#Перевірка статиcтичних гіпотез. Z-test.



0. Зчитайте дані з `data.csv` у змінну data, яка має тип pandas.DataFrame. Ми будемо далі працювати з цією змінною.

In [7]:
import pandas as pd


FILE_PATH = "/Users/nataliapodziolkina/Documents/data/data.csv"
data = pd.read_csv(FILE_PATH)   

print(data.head())
print(data.shape)




   Unnamed: 0       data
0           0  17.499453
1           1  19.662399
2           2   7.182823
3           3  29.841625
4           4   9.239386
(100, 2)


Запустіть код нижче. Для коректної робити всіх подальших методів дані мають бути у вигляді одновимірного `numpy.ndarray` вектора та мати розмірність (100,). Така розмірність означає, що дані - одновимірні (колонка), якщо б розмірність була (100, 1), то дані сприймаються методами як двовимірні (таблиця), хоча для нас виглядати можуть так само.

In [8]:
data = data.data.values
data.shape

(100,)

**Завдання 1**. Зчитані дані - це сума покупок в доларах наших покупців на сайті протягом липня 2025 року.

До цього місяця, ми мали продажі в середньому на чек $20.

Необхідно зʼясувати, чи вийшло у нас статистично значущо **підвищити** середній чек за липень 2025?

Аби дати відповідь - ми проведемо z-test на рівні значущості $\alpha=0.05$ двома способами. В цьому завданні ми будемо виконувати обчислення "вручну" з використанням бібліотек numpy та scipy.stats подібно до прикладу в лекції. Для цього виконайте наступні 7 кроків. Правильне виконання кожного з кроків оцінюється в 1 бал.

1. Запишіть параметри задачі у змінні Python:
    - `sample_mean` - $\bar{x}$, середнє значення в вибірці
    - `population_mean` - $\mu_0$, середнє значення в популяції (тобто те, з яким ми порівнюємо середнє вибіркове значення)
    - `population_std` - $\sigma$, вибіркове стандартне відхилення, яке ми вважаємо, що є рівним ст. відх. популяції, адже маємо достатньо велику вибірку.
    - `sample_size` - $n$, розмір вибірки
    - `alpha` - $\alpha$ рівень значущості
    
    **Увага!** Для обчислення стандартного відхилення маємо скористатись функцією `np.std(your_dataframe, ddof=1)`. Чому так - розбираємо в лекції про t-test.


2. Визначте для цієї задачі:
    - якою є гіпотеза $H_0$
    - якою є альтернативна гіпотеза $H_a$
    - з яким типом тесту ми маємо справу - лівосторонній, правосторонній чи двосторонній.

3. Обчисліть стандартну помилку SE.
4. Розрахуйте z-статистику.
5. Знайдіть критичне z-значення з допомогою бібліотеки stats.
6. Обчисліть p-value з допомогою бібліотеки stats.
7. Прийміть рішення, чи відхиляєте ви гіпотезу $H_0$. Для прийняття рішення зробіть і порівняння z-статистики з критичним значенням, і проаналізуйте p-value.

In [9]:
import numpy as np
from scipy import stats

# 1. Основні параметри для перевірки гіпотези
sample_mean = np.mean(data)                  # середнє значення у вибірці
population_mean = 20                         # середній чек до липня
population_std = np.std(data, ddof=1)        # стандартне відхилення вибірки (ddof=1)
sample_size = len(data)                      # розмір вибірки
alpha = 0.05                                 # рівень значущості

print("Sample mean:", round(sample_mean, 3))
print("Population std:", round(population_std, 3))
print("Sample size:", sample_size)

# 2. Гіпотеза:
# H0: μ = 20  → середній чек не змінився
# H1: μ > 20  → середній чек збільшився
# Тип тесту: правосторонній

# 3. Стандартна помилка
SE = population_std / np.sqrt(sample_size)

# 4. Z-статистика
z_stat = (sample_mean - population_mean) / SE

# 5. Критичне Z-значення для правостороннього тесту
z_crit = stats.norm.ppf(1 - alpha)

# 6. p-value
p_value = 1 - stats.norm.cdf(z_stat)

# 7. Висновок
print("\nZ-statistic:", round(z_stat, 3))
print("Critical Z-value:", round(z_crit, 3))
print("p-value:", round(p_value, 4))

if z_stat > z_crit and p_value < alpha:
    print("\n✅ Відхиляю H₀: середній чек статистично значуще збільшився.")
else:
    print("\n❌ Не відхиляю H₀: підвищення середнього чеку не є статистично значущим.")


Sample mean: 19.378
Population std: 6.254
Sample size: 100

Z-statistic: -0.995
Critical Z-value: 1.645
p-value: 0.8402

❌ Не відхиляю H₀: підвищення середнього чеку не є статистично значущим.


**Завдання 2.** Виконайте обчислення z-test з використанням бібліотеки statsmodels.

Отримайте z-статистику та р-значення.

Виведіть p-значення та зробіть висновок, чи ми маємо достатньо доказів, аби стверджувати, що середній чек зріс.

Чи зійшлись значення z-статистику та р-значення в цьому завданні з попередніми обчисленнями?

Завдання оцінюється в 3 бали.

In [10]:
from statsmodels.stats.weightstats import ztest

# Альтернативна гіпотеза — середній чек більший за 20 
z_stat2, p_value2 = ztest(data, value=20, alternative='larger')

print("Z-statistic (statsmodels):", round(z_stat2, 3))
print("p-value (statsmodels):", round(p_value2, 4))

# Висновок 
if p_value2 < 0.05:
    print("\n✅ Відхиляю H₀: середній чек статистично значуще зріс.")
else:
    print("\n❌ Не відхиляю H₀: немає достатньо доказів, що середній чек зріс.")

# Перевірка, чи зійшлись результати з ручними обчисленнями
print("\nПорівняння з попереднім завданням:")
print(f"Z з обчислень вручну = {round(z_stat, 3)}, зі statsmodels = {round(z_stat2, 3)}")
print(f"P-value вручну = {round(p_value, 4)}, зі statsmodels = {round(p_value2, 4)}")


Z-statistic (statsmodels): -0.995
p-value (statsmodels): 0.8402

❌ Не відхиляю H₀: немає достатньо доказів, що середній чек зріс.

Порівняння з попереднім завданням:
Z з обчислень вручну = -0.995, зі statsmodels = -0.995
P-value вручну = 0.8402, зі statsmodels = 0.8402
